# Mask R-CNN - Inspect Validation on ResNet101
Inspect and visualize validation code.

In [ ]:
import os
import sys
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon
from pycocotools.coco import COCO

# Root directory of the project
ROOT_DIR = os.path.abspath("./")
LOGS_DIR = os.path.join(ROOT_DIR, "logs")

# Import Mask RCNN
sys.path.append("Mask_RCNN")  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

%matplotlib inline

## Configurations

In [ ]:
import utility_poles as up
class InferenceConfig(up.UtilityPoleConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
config = InferenceConfig()
UP_DIR = os.path.join(ROOT_DIR, 'data_dataset')

## Model, Weights and Dataset

In [ ]:
# Create model
model = modellib.MaskRCNN(mode="inference", config=config, model_dir=LOGS_DIR)
# Load weights
weights_path = os.path.join(ROOT_DIR, 'mask_rcnn_poles_resnet101.h5')
model.load_weights(weights_path, by_name=True)

# Load dataset
if config.NAME == "poles":
    dataset = up.UtilityPoleDataset()
    dataset.load_pole(UP_DIR, "val")

# Must call before using the dataset
dataset.prepare()

## COCO Evaluation
Measuring the accuracy of object detectors in AP.

In [ ]:
cocoGt=COCO("{}/val/annotations.json".format(UP_DIR))
cocoEval, cocoResults = up.evaluate_coco(model, dataset, cocoGt)


In [ ]:
cocoResults = sorted(cocoResults, key=lambda i: i['score'], reverse=True)
class_ids = [i['category_id'] for i in cocoResults]
image_ids = np.array([i['image_id'] for i in cocoResults])
coco_bboxes = [i['bbox'] for i in cocoResults]
bboxes = np.array([])
rois = np.array([])
class_ids = np.array([])
scores = np.array([[]])
masks = np.array([[]])
ious = np.array([])

# Detect
for i, image_id in enumerate(dataset.image_ids):
    gt_masks, gt_class_ids = dataset.load_mask(image_id)
    gt_bboxes = utils.extract_bboxes(gt_masks)
    
    image = dataset.load_image(image_id)
    pred = model.detect([image], verbose=1)[0]
    rois = np.append(rois, pred['rois'])
    class_ids = np.append(class_ids, pred['class_ids'])
    scores = np.append(scores, pred['scores'])
    
    if pred['rois'].shape[0] > 0:
        overlaps = utils.compute_overlaps(pred['rois'], gt_bboxes)
        overlaps = np.amax(overlaps, axis=1)
        ious = np.append(ious, overlaps)
        
    visualize.display_differences(image, gt_bboxes, gt_class_ids, gt_masks,
                                  pred['rois'], pred['class_ids'], pred['scores'], pred['masks'],
                                  dataset.class_names)

## Precisions and Recalls

In [ ]:
score_ind = np.argsort(-scores)

from prettytable import PrettyTable

pred_gt_table = PrettyTable(['IoU', 'Score', ])
for i, ind in enumerate(score_ind):
    pred_gt_table.add_row(['{:.2f}'.format(ious[ind]), '{:.2f}'.format(scores[ind])])
print(pred_gt_table)
